In [47]:
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import os
import glob
import time
import datetime
import seaborn as sn
import heapq

In [2]:
index_choice = 'sp500'

In [3]:
SMALL_SIZE = 6
MEDIUM_SIZE = 8
BIGGER_SIZE = 10

plt.rc('font', size=MEDIUM_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=BIGGER_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=MEDIUM_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)

In [4]:
file_path = os.path.join("temp",f"{index_choice}",f"{index_choice}_final_smoothed.csv")
final_df = pd.read_csv(file_path)
final_df["datetime"] = final_df["date"].apply(lambda s: datetime.datetime.strptime(s, "%Y-%m-%d"))

In [5]:
def normalize (df):
    min = df.min()
    max = df.max()
    x = df
    y = (x-min) / (max-min)
    return y

In [6]:
industry_choice = ["Airlines","Internet Retail", "Electronic Gaming & Multimedia"]

industry_extract = final_df[final_df["industry"].isin(industry_choice)]

industry_extract["normalized_open"]= industry_extract.groupby("ticker")["open_x"].apply(lambda x: normalize (x))

industry_extract_grouped = industry_extract.groupby(["datetime","industry"]).mean("normalized_open").reset_index()

In [12]:
final_df["normalized_open"]= final_df.groupby("ticker")["open_x"].apply(lambda x: normalize (x))

final_df["change_open"]= (final_df.groupby("ticker")["open_x"].apply(pd.Series.pct_change))

window_size= 7
final_df["weekly_rolling_avg_open"]= final_df.groupby("ticker").rolling(window=window_size, min_periods= 1)['change_open'].mean().reset_index(drop=True)

window_size= 30
final_df["monthly_rolling_avg_open"]= final_df.groupby("ticker").rolling(window=window_size, min_periods= 1)['change_open'].mean().reset_index(drop=True)

open_by_stock = final_df.pivot(index="datetime", columns="ticker", values="normalized_open")

final_df.head()

,date,open_x,high,low,close,adjclose,volume_x,ticker,zip,sector,...,address2,underlyingSymbol,underlyingExchangeSymbol,headSymbol,uuid,datetime,change_open,weekly_rolling_avg_open,monthly_rolling_avg_open,normalized_open
0,2019-01-02,31.459999,32.650002,31.049999,32.480000,31.963160,5229500.0,AAL,76155,Industrials,...,NaN,NaN,NaN,NaN,NaN,2019-01-02,NaN,NaN,NaN,0.809201
1,2019-01-03,31.690001,31.850000,28.809999,30.059999,29.581665,16822000.0,AAL,76155,Industrials,...,NaN,NaN,NaN,NaN,NaN,2019-01-03,0.007311,-0.014586,-0.014586,0.817404
2,2019-01-04,30.440001,32.090000,30.400000,32.040001,31.530161,9369600.0,AAL,76155,Industrials,...,NaN,NaN,NaN,NaN,NaN,2019-01-04,-0.039445,-0.018281,-0.018281,0.772825
3,2019-01-07,31.990000,33.480000,31.240000,32.950001,32.425678,8010700.0,AAL,76155,Industrials,...,NaN,NaN,NaN,NaN,NaN,2019-01-07,0.050920,-0.004125,-0.004125,0.828103
4,2019-01-08,33.360001,33.650002,32.200001,32.419998,31.904110,7107700.0,AAL,76155,Industrials,...,NaN,NaN,NaN,NaN,NaN,2019-01-08,0.042826,0.004333,0.004333,0.876961


In [19]:
covid_usa = pd.read_csv("covid_usa_smoothed.csv")

covid_usa["datetime"] = covid_usa["date"].apply(lambda s: datetime.datetime.strptime(s, "%d/%m/%Y"))

#covid_usa.info()

final_df_covid = final_df.merge(covid_usa, how = "left", left_on= "datetime", right_on="datetime")

final_df_covid.tail(20)

,date_x,open_x,high,low,close,adjclose,volume_x,ticker,zip,sector,...,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,change_new_cases,rolling_avg_new_cases
219966,2020-09-02,162.100006,165.820007,161.820007,165.330002,165.123322,1790200.0,ZTS,07054,Healthcare,...,151.089,10.79,19.1,24.6,NaN,2.77,78.86,0.924,0.318730,0.034600
219967,2020-09-03,165.279999,165.520004,157.369995,158.800003,158.601486,1794100.0,ZTS,07054,Healthcare,...,151.089,10.79,19.1,24.6,NaN,2.77,78.86,0.924,-0.131835,-0.001956
219968,2020-09-04,159.160004,160.210007,155.289993,157.240005,157.043442,1552500.0,ZTS,07054,Healthcare,...,151.089,10.79,19.1,24.6,NaN,2.77,78.86,0.924,-0.064638,-0.021395
219969,2020-09-08,156.229996,156.270004,152.500000,153.509995,153.318100,1829100.0,ZTS,07054,Healthcare,...,151.089,10.79,19.1,24.6,NaN,2.77,78.86,0.924,-0.206349,-0.016955
219970,2020-09-09,155.199997,160.699997,155.199997,158.910004,158.711349,1408600.0,ZTS,07054,Healthcare,...,151.089,10.79,19.1,24.6,NaN,2.77,78.86,0.924,0.118433,-0.045569
219971,2020-09-10,158.899994,161.520004,157.210007,158.100006,157.902374,1356800.0,ZTS,07054,Healthcare,...,151.089,10.79,19.1,24.6,NaN,2.77,78.86,0.924,0.177162,-0.001426
219972,2020-09-11,159.000000,159.619995,157.850006,158.589996,158.391739,854100.0,ZTS,07054,Healthcare,...,151.089,10.79,19.1,24.6,NaN,2.77,78.86,0.924,0.174774,0.032776
219973,2020-09-14,159.940002,161.970001,159.610001,160.860001,160.658905,2092000.0,ZTS,07054,Healthcare,...,151.089,10.79,19.1,24.6,NaN,2.77,78.86,0.924,-0.170235,0.032073
219974,2020-09-15,162.250000,163.910004,161.699997,162.279999,162.077133,1039900.0,ZTS,07054,Healthcare,...,151.089,10.79,19.1,24.6,NaN,2.77,78.86,0.924,0.028638,0.065642
219975,2020-09-16,163.520004,163.990005,161.130005,161.250000,161.048416,1721200.0,ZTS,07054,Healthcare,...,151.089,10.79,19.1,24.6,NaN,2.77,78.86,0.924,0.477369,0.116919


In [42]:
### correlation matrix between new cases and open_x (absolute values)

df_corr_stock = final_df_covid.pivot(index="datetime", columns="ticker", values="normalized_open")

df_corr_covid = final_df_covid.pivot(index="datetime", columns="ticker", values="new_cases")

#df_corr_stock["AAPL"].corr(df_corr_covid["AAPL"], method="pearson")

correlation_list= {}

for ticker in df_corr_stock:
    correlation = df_corr_stock[ticker].corr(df_corr_covid[ticker], method="pearson")
    correlation_list[ticker]=correlation

print(correlation_list)

{'A': 0.5597352344556783, 'AAL': -0.6671031096203516, 'AAP': 0.2839973419613668, 'AAPL': 0.6203982148263543, 'ABBV': 0.46985792834700185, 'ABC': 0.6877297519565522, 'ABMD': 0.7801437174235112, 'ABT': 0.6473837133718365, 'ACN': 0.46363299521459034, 'ADBE': 0.6945416860156245, 'ADI': 0.22749812660201868, 'ADM': 0.09473312814396474, 'ADP': -0.5180234612405967, 'ADSK': 0.6195200185397297, 'AEE': -0.1185661580987958, 'AEP': -0.5609238033226198, 'AES': -0.27796637501439664, 'AFL': -0.5740340554065954, 'AIG': -0.5571624724934298, 'AIV': -0.6562161454699647, 'AIZ': -0.3671375080442624, 'AJG': 0.23983508788008664, 'AKAM': 0.8203095590702377, 'ALB': 0.26558969737523985, 'ALGN': 0.4236607292976052, 'ALK': -0.5793878106158965, 'ALL': -0.5783527452898511, 'ALLE': -0.5643066776882067, 'ALXN': 0.32608065402333425, 'AMAT': 0.23403191290387174, 'AMCR': 0.32536809224848395, 'AMD': 0.6057550048510592, 'AME': 0.09658269667295552, 'AMGN': 0.6287030077205353, 'AMP': 0.014264602836634726, 'AMT': 0.5376511364

In [63]:
### finding the max and min correlation values

###strongest positive correlation
print(heapq.nlargest(5,correlation_list,key=correlation_list.get))

###strongest negative correlation
print(heapq.nsmallest(5,correlation_list,key=correlation_list.get))

### weakest correlation
correlation_list_abs={}
for ticker in correlation_list:
    correlation_list_abs[ticker]=abs(correlation_list[ticker])

print(heapq.nsmallest(5,correlation_list_abs,key=correlation_list.get))


['CLX', 'NFLX', 'REGN', 'AMZN', 'ATVI']
['CMCSA', 'MCD', 'JCI', 'WMB', 'PPG']
['CMCSA', 'MCD', 'JCI', 'WMB', 'PPG']


In [74]:

df_corr_joint = final_df_covid.pivot(index="datetime", columns="ticker", values=["open_x","new_cases"])



KeyError: False